In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
hc_id_map = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/2_2_COCONUT_herb_compound_ID(remove_duplicate)_tab.txt")

In [3]:
sider = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_3_herb_sider_umlsID.tsv")
offs = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/6_3_herb_offside_umlsID.tsv")

In [7]:
# gold drug id를 알려고
gold_se_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/1_drug_drug_se_sim_mat.tsv", index_col=0)

In [5]:
# gold drug에 사용된 se 정보
gold_se_mat = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/4_2_gold_se_by_indep_matrix.tsv", index_col = 0)

In [8]:
gold_dr = gold_se_sim.columns # gold drug id
print len(gold_dr)

434


In [7]:
gold_se = gold_se_mat.index
len(gold_se_mat.index) # gold drug의 feature 수

1844

In [8]:
print len(set(sider['compID'].tolist())), len(set(sider['umls_id'].tolist())) # sider 정보를 가지는 compound의 수, se의 수
print len(set(offs['compID'].tolist())), len(set(offs['umls_id'].tolist())) # offside 정보를 가지는 compound의 수, adr의 수

113 1406
61 4516


In [9]:
si_off = pd.concat([sider, offs]).drop_duplicates() # sider와offside의 정보를 합쳐서 교집합 -> 공통 se feature를 뽑는것.
print len(si_off), len(si_off.drop_duplicates(['compID', 'umls_id']))
si_off.head(1)

28711 28711


,compID,compName,umls_id
0,5525,Benzyl alcohol,C0013604


In [10]:
si_off2 = si_off[si_off['umls_id'].isin(gold_se)] # gold에서 쓰인 se와의 교집합.

In [11]:
print len(set(si_off2['compID'])), len(set(si_off2['umls_id'])) # gold feature를 가지는 compound의 수, 교집합인 se의 수
si_off2.head(1)

135 1668


,compID,compName,umls_id
0,5525,Benzyl alcohol,C0013604


In [18]:
herb_se_mat = pd.DataFrame(columns=set(si_off2.compID.tolist()), index = gold_se.tolist())
herb_se_mat.fillna(0, inplace=True)

In [19]:
f = lambda x: 1 if x else 0
for compID in set(si_off2.compID.tolist()):
    temp = si_off2[si_off2.compID == compID]
    flags = gold_se.isin(temp['umls_id'])
    oz_flags = map(f, flags)
    herb_se_mat[compID] = oz_flags

In [20]:
# herbal compound와 gold drug 간의 se similarity score 구하기
from sklearn.metrics import jaccard_similarity_score
hc_id = set(si_off2.compID.tolist())
df_jac = pd.DataFrame(columns=gold_dr.tolist(), index = hc_id)
df_jac.fillna(0.0, inplace = True)
for i in gold_dr:
    desc1 = gold_se_mat[i]
    for j in hc_id:
        desc2 = herb_se_mat[j]
        
        jac_sim = jaccard_similarity_score(desc1, desc2)
        df_jac.set_value(j, i, jac_sim)

In [ ]:
# herbal compound 중 gold drug와 겹치는 것이 있을 것이다. 제외해야함.(기존 disease와의 연관을 볼때 사용)

In [12]:
ext = hc_id_map[hc_id_map['compID'].isin(si_off2['compID'])] # 내가 알고싶은 herbal compound의 외부 id 정보
ext.head(1)

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
69,"[1, 3, 127, 175, 247, 254, 326, 357, 383, 489,...","['Melia azedarach', 'Paeonia lactiflora', 'Cam...",5525,Benzyl alcohol,null,100-51-6,null,null


In [13]:
# 1차는 stitch id로 거른다.
gold_id_map = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_3_SIDER_STITCHid_DrugBankid.tsv") # gold drug의 stitch id

In [14]:
# gold_id_map에 있는 stitch id 정보 중 herbal compound의 stitch id가 있는지 본다.
#gold_dr[gold_dr.isin(gold_id_map[gold_id_map['STITCH_compound_id_stereo'].isin(ext['stitchID'])]['DrugBank_ID'])]

# herbal compound의 stitch id로 DrugBank ID를 찾는다. 
remv = ext[ext.stitchID.isin(gold_id_map[gold_id_map.DrugBank_ID.isin(gold_dr)].STITCH_compound_id_stereo)]
print len(remv)
remv.head(1)

11


,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
2623,"[73, 636]","['Cannabis sativa', 'Juanulloa']",70313,delta(9)-tetrahydrocannabinol,16078,null,null,CID000016078


In [23]:
# stitch_id로 거른것.
filtered1 = df_jac[~df_jac.index.isin(remv.compID)]
print len(filtered1)
filtered1.head(1)

124


,DB00114,DB00126,DB00136,DB00153,DB00162,DB00165,DB00175,DB00176,DB00177,DB00178,...,DB01576,DB01577,DB01590,DB01623,DB02546,DB03904,DB04272,DB04844,DB04861,DB09026
70661,0.933839,0.996746,0.957701,0.986985,0.997831,0.755423,0.933839,0.895879,0.956616,0.91974,...,0.954447,0.990781,0.91757,0.966377,0.91974,0.939805,0.990239,0.934924,0.913774,0.940889


In [22]:
filtered_ext1 = ext[ext.compID.isin(filtered1.index)]
print len(filtered_ext1)
filtered_ext1.head(1)

124


,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
69,"[1, 3, 127, 175, 247, 254, 326, 357, 383, 489,...","['Melia azedarach', 'Paeonia lactiflora', 'Cam...",5525,Benzyl alcohol,null,100-51-6,null,null


In [24]:
#unichem을 통해 pc --> DB 해야하는 herbal compounds들.
query_pc = filtered_ext1[(filtered_ext1.pubchemID != "null") & (filtered_ext1.stitchID == "null")]

In [25]:
print filtered_ext1[(filtered_ext1.pubchemID != "null") & (filtered_ext1.stitchID == "null")].shape
print filtered_ext1[(filtered_ext1.pubchemID == "null") & (filtered_ext1.stitchID == "null") & (filtered_ext1.chemblID != "null")].shape
print filtered_ext1[(filtered_ext1.pubchemID == "null") & (filtered_ext1.stitchID == "null") & (filtered_ext1.casID != "null")].shape

(66, 8)
(1, 8)
(35, 8)


In [26]:
query_pc.head(1)

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
399,"[3, 314, 336, 792, 3227, 3838, 4161, 4689, 492...","['Paeonia lactiflora', 'Crocus sativus', 'Isat...",1020,Probenecid,4911,57-66-9,CHEMBL897,null


In [100]:
# using UniChem REST API, convert pubchemID to DrugBank ID
from urllib2 import urlopen # python2
import urllib2
import time
df_result = pd.DataFrame()
cnt = 0
url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
print'%d : ' % cnt + time.strftime('%a %H:%M:%S')
for index, row in query_pc.iterrows():
    pc_id = row.pubchemID
    url = url_base + str(pc_id) + '/22/2'
    
    MAX_ATTEMPTS = 8
    for attempt in range(MAX_ATTEMPTS):
        try:
            f = urlopen(url)
        except urllib2.HTTPError, e:
            if e.code == 404:
                print 'HTTPError 404 not found : compID={}'.format(row.compID) + time.strftime('%a %H:%M:%S')
                cnt = cnt + 1
                if cnt % 1000 == 0 :
                    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
                continue
            else:
                print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                raise
        except urllib2.URLError, e:
            if e.args[0][0] == 110: # connection time out
                sleep_secs = attempt ** 2
                print e, url
                print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                print '    Retrying in %d seconds...' % sleep_secs
                time.sleep(sleep_secs)
                cnt= cnt +1
                if cnt % 1000 == 0 :
                    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
                continue 
            else:
                print e
                print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                raise
        else:          
            all_data = f.readlines()
            if all_data[0] == '[]':
                cnt = cnt + 1
            else:
                #db_id=all_data[0].decode("utf-8").split(':')[1]  
                db_id=all_data[0]
                temp_df = pd.DataFrame(columns=['compID', 'PubChemID', 'DrugBankID'], data = [[row.compID, pc_id, db_id]])
                df_result = df_result.append(temp_df)
                cnt = cnt + 1
            if cnt % 1000 == 0 :
                print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')


0 : Fri 00:57:55
HTTPError 404 not found : compID=79944Fri 00:59:20
HTTPError 404 not found : compID=79944Fri 00:59:21
HTTPError 404 not found : compID=79944Fri 00:59:22
HTTPError 404 not found : compID=79944Fri 00:59:23
HTTPError 404 not found : compID=79944Fri 00:59:24
HTTPError 404 not found : compID=79944Fri 00:59:25
HTTPError 404 not found : compID=79944Fri 00:59:26
HTTPError 404 not found : compID=79944Fri 00:59:27
HTTPError 404 not found : compID=79938Fri 00:59:37
HTTPError 404 not found : compID=79938Fri 00:59:38
HTTPError 404 not found : compID=79938Fri 00:59:39
HTTPError 404 not found : compID=79938Fri 00:59:40
HTTPError 404 not found : compID=79938Fri 00:59:41
HTTPError 404 not found : compID=79938Fri 00:59:42
HTTPError 404 not found : compID=79938Fri 00:59:43
HTTPError 404 not found : compID=79938Fri 00:59:44
Fri 01:07:15


In [124]:
df_result = df_result.drop_duplicates()
df_result.head(1)

,compID,PubChemID,DrugBankID
0,1020,4911,"[{""src_compound_id"":""DB01032""}]"


In [107]:
# df_result2는 compoundID와 PubChemID, DrugBankID가 1:1로 매핑되어 있는것.
df_result2 = pd.DataFrame()
for index, row in df_result.iterrows():
    record = row.tolist()
    db_ids = record[2].split(',')
    for i in db_ids:
        db_id = i.split(':')[1].lstrip('\"').rstrip('\"}]')
        temp_df = pd.DataFrame([[record[0], record[1], db_id]], columns = ['compID', 'PubChemID', 'DrugBankID'])
        df_result2 = df_result2.append(temp_df)
df_result2.index = range(len(df_result2))

In [132]:
# pubchemID로 converting한 DrugBank id가 gold drug에 포함되어 있으면 제외한다.
print len(gold_dr[gold_dr.isin(df_result2.DrugBankID)])# gold drug와 겹치는 herbal compound들.
print gold_dr[gold_dr.isin(df_result2.DrugBankID)]

filtered_ext2 = filtered_ext1[~filtered_ext1.compID.isin(df_result2[df_result2.DrugBankID.isin(gold_dr)].compID)]
print len(filtered_ext2)
filtered_ext2.head(1)

14
Index([u'DB00126', u'DB00162', u'DB00318', u'DB00368', u'DB00477', u'DB00482',
       u'DB00536', u'DB00627', u'DB00640', u'DB00829', u'DB00917', u'DB01113',
       u'DB01356', u'DB01378'],
      dtype='object')
110


,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
69,"[1, 3, 127, 175, 247, 254, 326, 357, 383, 489,...","['Melia azedarach', 'Paeonia lactiflora', 'Cam...",5525,Benzyl alcohol,null,100-51-6,null,null


In [143]:
print len(filtered_ext2[(filtered_ext2.pubchemID == "null") & (filtered_ext2.stitchID == "null") & (filtered_ext2.casID != "null")])
filtered_ext2[(filtered_ext2.pubchemID == "null") & (filtered_ext2.stitchID == "null") & (filtered_ext2.casID != "null")].head(2)

35


,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
69,"[1, 3, 127, 175, 247, 254, 326, 357, 383, 489,...","['Melia azedarach', 'Paeonia lactiflora', 'Cam...",5525,Benzyl alcohol,null,100-51-6,null,null
763,"[9, 91, 124, 336, 457, 547, 630, 653, 681, 982...","['Trogopterus xanthipes', 'Glycine max', 'Jugl...",926,Salicylic acid,null,69-72-7,null,null


In [135]:
# a는 casID로 drugBank ID를 찾은거.
a = ["DB06770", "DB00936", "DB00165", "DB01592", "DB01593", "DB00159", "DB00548", "DB00162", "DB04816", "DB01956", "DB00632", "DB00988",
     "DB00553", "DB00741", "DB01115", "DB03128", "DB00572", "DB00553", "DB00945", 'DB01179', "DB00572", "DB08826", "DB02821", "DB00755",
     "DB00397", "DB00762", "DB01180", 'DB00147', "DB01373", "DB08117", 'DB01591', 'DB00686']

In [121]:
# casID로 바꾼 drugBank ID 중 gold drug에 포함되는 id들. --> 이중 DB00762는 잘못된 convert임.
filtered_ext2[~filtered_ext2.compID.isin(df_result2.compID)]

Index([u'DB00162', u'DB00165', u'DB00397', u'DB00548', u'DB00686', u'DB00755',
       u'DB00762', u'DB00945', u'DB00988', u'DB01115'],
      dtype='object')

In [136]:
# 위의 포함된 drugbank ID에 해당되는 casID들
rmv_cas_id = ['65-23-6', '123-99-9', '68-26-8', '51-61-6', '21829-25-4', '50-78-2', '302-79-4', '14838-15-4', '37300-21-3']

In [140]:
# gold drug(cas_ID로 찾은) 제외한 compound들
filtered_ext3 = filtered_ext2[~filtered_ext2.casID.isin(rmv_cas_id)]

In [83]:
filtered_ext1[(filtered_ext1.pubchemID == "null") & (filtered_ext1.chemblID != "null")] --> 얘의 drugbank id는 DB00572 이고, gold drug와 겹치지 않는다.

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
8315,"[369, 653, 826]","['Arctium lappa', 'Lycium barbarum', 'Hyoscyam...",562,Atropine,null,51-55-8,CHEMBL517712,null


In [142]:
filtered_ext3[(filtered_ext3.pubchemID == "null") & (filtered_ext3.chemblID != "null")] --> 얘의 drugbank id는 DB00572 이고, gold drug와 겹치지 않는다.

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
8315,"[369, 653, 826]","['Arctium lappa', 'Lycium barbarum', 'Hyoscyam...",562,Atropine,null,51-55-8,CHEMBL517712,null


In [148]:
hc_dr_se_sim = df_jac.filter(items = filtered_ext3.compID, axis=0)

In [150]:
print len(hc_dr_se_sim)
hc_dr_se_sim.head(1)

101


,DB00114,DB00126,DB00136,DB00153,DB00162,DB00165,DB00175,DB00176,DB00177,DB00178,...,DB01576,DB01577,DB01590,DB01623,DB02546,DB03904,DB04272,DB04844,DB04861,DB09026
5525,0.930586,0.990239,0.956616,0.981562,0.991323,0.754338,0.932755,0.893709,0.954447,0.918655,...,0.949024,0.984273,0.914317,0.963124,0.914317,0.933297,0.984816,0.928416,0.91269,0.937636


In [192]:
# strucutre smiles를 찾을 수 없는 compound를 지운다.
hc_dr_se_sim.drop(88531, inplace = True)
df_jac.drop(88531, inplace=True)

In [193]:
df_jac.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_1_1_herbal_comp_gold_drug_se_sim_mat_validation.tsv", sep='\t')
hc_dr_se_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", sep='\t')

In [11]:
df_jac = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_1_1_herbal_comp_gold_drug_se_sim_mat_validation.tsv", index_col = 0)
hc_dr_se_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col = 0)

In [ ]:
# structure similarity 구하기

In [3]:
drugbank = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/drug_set.txt", header=None)
drugbank = drugbank.rename(columns = {0:'DrugBank_ID', 1:'Drug_Name', 2:'InChI', 3:'SMILES', 4:'State'})
drugbank.head(1)

,DrugBank_ID,Drug_Name,InChI,SMILES,State
0,DB00006,Bivalirudin,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,approved


In [4]:
hc = pd.read_csv("/DAS_Storage1/aschoi/data/COCONUT_Herbal/6_Herb/1_3_COCONUT_herb_compound(remove duplicate).csv")

In [5]:
hc

,compID,compName,smiles,InChI,herbID,name
0,39863,Azedarachin A,CCC(C)C(=O)OC1[C@@]2([C@@H]3C[C@H]([C@@]4([C@@...,InChI=1/C33H44O11/c1-7-15(2)27(39)43-28-29(4)1...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
1,39879,"(1S,2R,8R,10R,11S,15R)-21-Acetoxy-6-(3-furyl)-...",CC(C)C(=O)OC1[C@]2(C3CC([C@@]4(C([C@]3(CO1)C(C...,InChI=1/C32H42O10/c1-15(2)26(37)41-27-29(5)20-...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
2,39880,"1-(3-Furyl)-5-(hydroxymethyl)-8a-methyl-1,7,8,...",CC12CCC=C(C1=CC(=O)OC2c3ccoc3)CO,InChI=1/C15H16O4/c1-15-5-2-3-10(8-16)12(15)7-1...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
3,39882,"Methyl [(1S,5S,7S,8S,16S)-5,11-diacetoxy-16-{[...",C[C@@H]1[C@@H]([C@H]([C@H]([C@@H](O1)OC[C@@H]2...,InChI=1/C43H60O20/c1-17-22-12-27(59-20(4)45)42...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
4,39883,Trichilin H,CC(C)C(=O)OC1[C@@]2([C@@H]3C[C@H]([C@@]4([C@@H...,InChI=1/C36H46O14/c1-15(2)30(43)49-31-32(6)21-...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
5,38197,"(5xi,9xi,13xi,14beta,17alpha,20S,23R,24S)-21,2...",C[C@]12CCC(=O)C(C1CC=C3C2CCC4([C@@]3(CC[C@H]4[...,"InChI=1/C30H48O5/c1-26(2)22-9-8-20-19(28(22,5)...","[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
6,38198,7-Tricosanol,CCCCCCCCCCCCCCCCC(CCCCCC)O,InChI=1/C23H48O/c1-3-5-7-9-10-11-12-13-14-15-1...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
7,38199,a-elemene,CC(C)C1=CC(=C(C)C)CC[C@@]1(C)C=C,InChI=1/C15H24/c1-7-15(6)9-8-13(11(2)3)10-14(1...,"[1.0, 168.0, 1909.0]","['Melia azedarach', 'Melia azed-arach', 'Artem..."
8,33578,Nimbolin B,CC(=O)O[C@@H]1C[C@@H]([C@@]2([C@H]3CC(O[C@H]4C...,InChI=1/C38H44O10/c1-21(39)45-29-18-30(46-22(2...,"[1.0, 168.0]","['Melia azedarach', 'Melia azed-arach']"
9,32837,Daucosterol,CC[C@H](CC[C@@H](C)[C@H]1CC[C@@H]2[C@@]1(CC[C@...,InChI=1/C35H60O6/c1-7-22(20(2)3)9-8-21(4)26-12...,"[1.0, 2.0, 17.0, 24.0, 52.0, 54.0, 76.0, 87.0,...","['Melia azedarach', 'Ailanthus altissima', 'Ar..."


In [9]:
gold_smiles = drugbank[drugbank.DrugBank_ID.isin(gold_dr)]

In [12]:
hc_smiles = hc[hc.compID.isin(hc_dr_se_sim.index)]

In [13]:
hc_smiles = hc_smiles[["compID", "smiles"]]
print len(hc_smiles)

99


In [14]:
# 원래 101개인데 정보가 없는 2개 확인
hc_dr_se_sim.index[~hc_dr_se_sim.index.isin(hc_smiles.compID)] #: 이중 88531은 smiles를 찾을 수가 없다.

Int64Index([40553], dtype='int64')

In [15]:
# 40553은 drugbank에서 compound 이름으로 먼저 검색하고, 나온 smiles로 chemspider에 검색해서 chemspider smiles를 얻었음.
hc_smiles = hc_smiles.append(pd.DataFrame(data= [[40553, "COc1c2c(ccc(=O)o2)cc3c1occ3"]], columns=["compID", 'smiles']))

In [27]:
hc_smiles[hc_smiles.compID == 80164]

,compID,smiles
4192,80164,[N]


In [41]:
import pybel
df_sim = pd.DataFrame(columns= gold_dr.tolist(), index = hc_smiles.compID.tolist())
df_sim.fillna(0.0, inplace=True)
for i in gold_dr.tolist():
    smi1 = gold_smiles[gold_smiles.DrugBank_ID == i]['SMILES'].tolist()[0]
    for j in hc_smiles.compID.tolist():
        smi2 = hc_smiles[hc_smiles.compID == j]['smiles'].tolist()[0]
        
        smiles = [smi1, smi2]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        
        df_sim.set_value(j, i, sim)

In [43]:
val_smiles = hc[hc.compID.isin(df_jac.index.tolist())]
val_smiles = val_smiles[['compID', 'smiles']]
val_smiles = val_smiles.append(pd.DataFrame(data= [[40553, "COc1c2c(ccc(=O)o2)cc3c1occ3"]], columns=["compID", 'smiles']))

In [44]:
df_sim2 = pd.DataFrame(columns= gold_dr.tolist(), index = df_jac.index.tolist())
df_sim2.fillna(0.0, inplace=True)
for i in gold_dr.tolist():
    smi1 = gold_smiles[gold_smiles.DrugBank_ID == i]['SMILES'].tolist()[0]
    for j in df_jac.index.tolist():
        smi2 = val_smiles[val_smiles.compID == j]['smiles'].tolist()[0]
        
        smiles = [smi1, smi2]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        
        df_sim2.set_value(j, i, sim)

In [50]:
df_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_2_herbal_comp_gold_drug_str_sim_mat_predict.tsv", sep='\t')
df_sim2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_1_2_herbal_comp_gold_drug_str_sim_mat_validation.tsv", sep='\t')